# TUTORIAL: py_to_psql & opendtect_to_py 

In [1]:
import psycopg2 as p
import py_to_psql as pp
import opendtect_to_py as op
import time 

## Database connection

In [2]:
conn = p.connect(
    host="192.168.1.107",
    database="tutorial",
    user="lsdr94",
    password="Pelusa120894"
)

## Table creation

In [3]:
pp.wells_table_creation("wells", conn)

'Command has been executed successfully in 0.11536598205566406s'

## Fetch query results

In [4]:
query = "SELECT * FROM wells LIMIT 1"
pp.fetch_psql_command(query, conn)

(['well_id',
  'opendtect_id',
  'well_name',
  'x_coordinate',
  'y_coordinate',
  'status'],
 [])

## Fetch column name of wells table

In [5]:
pp.fetch_column_names("wells", conn)

['well_id',
 'opendtect_id',
 'well_name',
 'x_coordinate',
 'y_coordinate',
 'status']

## Print opendtect wells info

In [6]:
# Print objects
for well in op.fetch_opendtect_wells_info():
    print(well)
    # Too many wells. Lets break the loop.
    break

{'ID': '100050.3', 'Name': '19/08- 1', 'Format': 'dGB', 'TranslatorGroup': 'Well', 'File_name': 'C:\\Users\\Monster\\Documents\\Opendtect\\MNSH_OGA_2016_v1\\WellInfo\\19_08-_1.well', 'Status': 'OK', 'X': 236383.46063858, 'Y': 6405003.51221513}


### Fetch opendtect well info

In [11]:
opdt_wells = op.fetch_opendtect_wells_info()

## Insert well information in table

In [12]:
init = time.time()

table_name = "wells"
table_col2 = "opendtect_id"
table_col3 = "well_name"
table_col4 = "x_coordinate"
table_col5 = "y_coordinate"
table_col6 = "status"
    
for well_dict in opdt_wells:
    init2 = time.time()
    well_upsertion_query = f"""
        INSERT INTO {table_name}({table_col2}, {table_col3}, {table_col4}, {table_col5}, {table_col6})
        VALUES ({well_dict['ID']}, '{well_dict['Name']}', {well_dict['X']}, {well_dict['Y']}, '{well_dict['Status']}')
        ON CONFLICT ({table_col3}) DO NOTHING;
    """      
    pp.execute_psql_command(well_upsertion_query, conn)
    end2 = time.time()
    print(f"Well {well_dict['Name']} has been inserted successfully in {end2 - init2}s\n")
        
end = time.time()
print(f"Wells insertion completed in {end - init}")

Well 19/08- 1 has been inserted successfully in 0.03130912780761719s

Well 19/15- 1 has been inserted successfully in 0.6718876361846924s

Well 20/06- 1 has been inserted successfully in 1.557173490524292s

Well 20/06- 2 has been inserted successfully in 0.015688419342041016s

Well 20/06- 3 has been inserted successfully in 0.07838845252990723s

Well 20/06- 3Z has been inserted successfully in 0.08502769470214844s

Well 20/06- 6 has been inserted successfully in 0.015639305114746094s

Well 20/12- 1 has been inserted successfully in 0.34427452087402344s

Well 20/12- 2 has been inserted successfully in 2.1557998657226562s

Well 20/12- 3 has been inserted successfully in 0.8761622905731201s

Well 21/21- 1 has been inserted successfully in 1.8300974369049072s

Well 21/26- 1D has been inserted successfully in 8.736373662948608s

Well 26/04- 1 has been inserted successfully in 0.015656709671020508s

Well 26/07- 1 has been inserted successfully in 0.015901565551757812s

Well 26/08- 1 has been

## Fetch Opendtect log

In [13]:
op.fetch_opendtect_well_log("19/08- 1", "Joined Well Logs`DTCO [D]")

([297.78961181640625,
  297.9419860839844,
  298.0943908691406,
  298.2467956542969,
  298.3992004394531,
  298.5516052246094,
  298.70401,
  298.8564147949219,
  299.0087890625,
  299.16119384765625,
  299.3135986328125,
  299.46600341796875,
  299.618408203125,
  299.77081298828125,
  299.9231872558594,
  300.0755920410156,
  300.2279968261719,
  300.3804016113281,
  300.5328063964844,
  300.6852111816406,
  300.83758544921875,
  300.99,
  301.14239501953125,
  301.2947998046875,
  301.44720458984375,
  301.599609375,
  301.75201416015625,
  301.9043884277344,
  302.0567932128906,
  302.2091979980469,
  302.3616027832031,
  302.5140075683594,
  302.6664123535156,
  302.81878662109375,
  302.97119140625,
  303.12359619140625,
  303.276,
  303.42840576171875,
  303.580810546875,
  303.7331848144531,
  303.8855895996094,
  304.0379943847656,
  304.1903991699219,
  304.3428039550781,
  304.4952087402344,
  304.6476135253906,
  304.8,
  304.952392578125,
  305.10479736328125,
  305.257202

## Log table creation

In [14]:
# Track table
track_columns = [
    "well_id INT NOT NULL UNIQUE REFERENCES wells(well_id),",
    "md_in_m NUMERIC(10,2)[] NOT NULL,",
    "tvdss_in_m NUMERIC(10,2)[] NOT NULL,",
    "x_coordinate NUMERIC(10,2)[] NOT NULL,",
    "y_coordinate NUMERIC(10,2)[] NOT NULL,",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("track_table", conn, column_list=track_columns)

# gr table
gr_columns = [
    "well_id INT NOT NULL UNIQUE REFERENCES wells(well_id),",
    "md_in_m NUMERIC(13,5)[],",
    "gr NUMERIC(13,5)[],",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("gr_table", conn, column_list=gr_columns)

'Command has been executed successfully in 0.09685492515563965s'

## Fetch well names

In [15]:
import odpy.wellman as wm
well_names = wm.getNames(reload=True)

## Log insertion

In [16]:
print("*"*25, "Track table", "*"*25)
op.insert_logs(well_names, "track", "track_table", "wells", conn)

************************* Track table *************************
Done inserting well 19/08- 1 'track' log.Execution time = 2.4413459300994873s

Done inserting well 19/15- 1 'track' log.Execution time = 2.3748974800109863s

Done inserting well 20/06- 1 'track' log.Execution time = 2.3707945346832275s

Done inserting well 20/06- 2 'track' log.Execution time = 2.3068573474884033s

Done inserting well 20/06- 3 'track' log.Execution time = 2.390538215637207s

Done inserting well 20/06- 3Z 'track' log.Execution time = 2.3986127376556396s

Done inserting well 20/06- 6 'track' log.Execution time = 2.36848521232605s

Done inserting well 20/12- 1 'track' log.Execution time = 2.3038346767425537s

Done inserting well 20/12- 2 'track' log.Execution time = 2.3309104442596436s

Done inserting well 20/12- 3 'track' log.Execution time = 2.348726511001587s

Done inserting well 21/21- 1 'track' log.Execution time = 2.3496875762939453s

Done inserting well 21/26- 1D 'track' log.Execution time = 2.305509328

"Log 'track' insertion completed in 233.29467582702637s"

In [17]:
print("*"*25, "GR table", "*"*25)
op.insert_logs(well_names, "Joined Well Logs`GR", "gr_table", "wells", conn)

************************* GR table *************************
Done inserting well 19/08- 1 'Joined Well Logs`GR' log.Execution time = 2.4396982192993164s

Done inserting well 19/15- 1 'Joined Well Logs`GR' log.Execution time = 2.452901840209961s

Done inserting well 20/06- 1 'Joined Well Logs`GR' log.Execution time = 2.7599892616271973s

Done inserting well 20/06- 2 'Joined Well Logs`GR' log.Execution time = 2.781437635421753s

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Joined Well Logs`GR']' returned non-zero exit status 1.
Log Joined Well Logs`GR not found for Well 20/06- 3.
Done inserting well 20/06- 3 'Joined Well Logs`GR' log.Execution time = 2.321157693862915s

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.217', 'Joined Well Logs`GR']' returned non-zero exit status 1.
Log Joined Well Logs`GR not found for Well 20/06- 3Z.
Done inserting well 

"Log 'Joined Well Logs`GR' insertion completed in 268.66188168525696s"

## COMPLETE TABLES

### TABLE ANALYSIS
#### GR TABLE
##### Find wells where gr log is NULL

In [18]:
gr_null_wells, gr_empty_wells = op.check_null_wells("gr", "gr_table", "wells", conn)


Well '20/06- 3' logs:
****************************************
Checkshots`One-way time 1[active]

Well '20/06- 3Z' logs:
****************************************
Checkshots`One-way time 1[active]

Well '29/06b- 5' logs:
****************************************
Checkshots`One-way time 1[active]

Well '29/11a- 1' logs:
****************************************
Raw CDA Logs`Deep Resistivity`ILD1
Raw CDA Logs`Medium Resistivity`ILM1
Raw CDA Logs`Micro Resistivity`SFLU1
Raw CDA Logs`DTCO [D]
Raw CDA Logs`Input to Dervied Logs`DT1
Raw CDA Logs`Input to Dervied Logs`GR1
Raw CDA Logs`Input to Dervied Logs`SP1
Raw CDA Logs`Input to Dervied Logs`CALI1
Raw CDA Logs`Input to Dervied Logs`TEN
Raw CDA Logs`GR [D]
Raw CDA Logs`SP [D]
Raw CDA Logs`CALI [D]
Raw CDA Logs`TEN [D]

Well '30/28- 2' logs:
****************************************
Raw CDA Logs`DTCO [D]
Raw CDA Logs`Input to Dervied Logs`DTCO1
Raw CDA Logs`Input to Dervied Logs`DTSM1
Raw CDA Logs`DTSH [D]

Well '31/27- 1' logs:
***************

In [19]:
print(gr_null_wells)
print(gr_empty_wells)

['20/06- 3', '20/06- 3Z', '21/26- 1D', '28/04a- 3', '28/15- 1', '29/06b- 5', '29/11a- 1', '29/19- 1A', '30/28- 2', '30/29a- 3', '31/27- 1', '37/12- 1', '39/01- 2', '39/02- 1', '39/02- 4', '41/01- 1', '41/14- 1', '41/18- 1', '41/18- 2', '41/20- 1', '41/25a- 1', '42/16- 1', '42/18- 2', '42/21- 1', '42/22- 1', '43/10- 1']
['21/26- 1D', '28/04a- 3', '28/15- 1', '29/19- 1A', '30/29a- 3', '39/02- 1', '41/18- 1', '41/18- 2', '41/20- 1', '41/25a- 1', '42/16- 1', '43/10- 1']


##### Store secondary gr log in gr_table

In [20]:
cnfct = "UPDATE SET md_in_m = EXCLUDED.md_in_m, "
cnfct += "gr = EXCLUDED.gr, log_name = EXCLUDED.log_name"

for well in gr_null_wells:
    op.insert_log(well, "Raw CDA Logs`GR [D]", "gr_table", "wells", conn, on_conflict_do=cnfct)

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Raw CDA Logs`GR [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`GR [D] not found for Well 20/06- 3.
Done inserting well 20/06- 3 'Raw CDA Logs`GR [D]' log.Execution time = 2.454294204711914s

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.217', 'Raw CDA Logs`GR [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`GR [D] not found for Well 20/06- 3Z.
Done inserting well 20/06- 3Z 'Raw CDA Logs`GR [D]' log.Execution time = 2.400177478790283s

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.221', 'Raw CDA Logs`GR [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`GR [D] not found for Well 21/26- 1D.
Done inserting well 21/26- 1D 'Raw CDA Logs`GR [D]' log.Execution time = 2.322052478790283s

Failed:  Command '['C:\\PROGRA~1\\OP